In [ ]:
https://learn.microsoft.com/en-us/azure/stream-analytics/stream-analytics-window-functions

#### Windowing Functions

In [ ]:
Ahora, en este capítulo, quiero ir a través de las "Windowing functions", que está disponible en stream analytics. Así que recuerda, el 
tiempo es el factor más importante cuando se trata de tus stream analytics jobs. Y normalmente en los escenarios en tiempo real, es posible 
que desee realizar cálculos en determinados window frames, por lo que puede hacer uso de las windowing functions para este propósito en 
particular. Ahora, hay diferentes "windowing functions". Tienes la "tumbling window", la "hopping window", la "sliding window", la 
"session window" y la "snapshot window". Así, por ejemplo, si nos fijamos en la "tumbling window", el propósito de esta ventana es realizar 
la agregación de los datos dentro de un marco de tiempo (time frame) particular. Así, cada duración de marco de tiempo constituye en 
realidad una ventana (window). Del mismo modo, es posible que desee realizar la agregación de algo durante períodos de tiempo, especialmente
cuando se trata de datos en tiempo real. A veces puede que no quieras toda la información, quieres realizar algún tipo de agregación sobre 
los datos dentro de un marco de tiempo determinado. 

<center><img src="https://i.postimg.cc/Ss3k4YPM/event2.png"></center>

In [ ]:
TUMBLING WINDOW
===============

Las funciones Tumbling window se utilizan para segmentar un stream de datos en distintos segmentos temporales y realizar una función 
sobre ellos, como en el ejemplo siguiente. Las principales características de una "Tumbling window" son que se repiten, no se sobreponen 
y que un evento no puede pertenecer a más de una "Tumbling window".

Así que aquí en realidad puedes usar la función tumbling window y decir, por favor agrega mis datos sobre un marco de tiempo de 10 
segundos. Así que aquí cada ventana es de 10 segundos de duración y, a continuación, puede utilizar sus funciones de agregación que 
ha requerido. 

<center><img src="https://i.postimg.cc/13gNSD49/event3.png"></center>

In [ ]:
Con los siguientes datos de entrada (ilustrados más arriba):

Stamp	CreatedAt	            TimeZone
1	    2021-10-26T10:15:01	    PST
5	    2021-10-26T10:15:03	    PST
4	    2021-10-26T10:15:06	    PST
...	    ...	                    ...

La siguiente consulta:

SELECT 
    System.Timestamp() as WindowEndTime, 
    TimeZone, 
    COUNT(*) AS Count
FROM 
    TwitterStream TIMESTAMP BY CreatedAt
GROUP BY 
    TimeZone, 
    TumblingWindow(second,10)

Se devolverá:

WindowEndTime	        TimeZone	Count
2021-10-26T10:15:10	    PST	        5
2021-10-26T10:15:20	    PST	        2
2021-10-26T10:15:30	    PST	        4

In [ ]:
Así que hay diferentes tipos de windows que están disponibles. Así, por ejemplo, también tenemos la "hopping window" en la que tenemos 
un escenario de caso de uso. Se dice que, durante un período de 10 segundos, cada cinco segundos, me da el número de, digamos, los 
tweets que están llegando. Así que, en la hopping window, puede haber una superposición de eventos. 

HOPPING WINDOW
==============

Las funciones "Hopping window" saltan hacia delante en el tiempo durante un periodo fijo. Puede ser fácil pensar en ellas como si fueran 
"Tumbling windows" que pueden superponerse y emitirse más a menudo que el tamaño de la ventana. Los eventos pueden pertenecer a más de un 
conjunto de resultados de una Hopping window. Para que una Hopping window sea igual que una Tumbling window, especifique que el tamaño 
del hop sea el mismo que el tamaño de la ventana.

<center><img src="https://i.postimg.cc/Y9sy0JQ4/event4.png"></center>

In [ ]:
Con los siguientes datos de entrada (ilustrados más arriba):

Stamp	CreatedAt	            Topic
1	    2021-10-26T10:15:01	    Streaming
5	    2021-10-26T10:15:03	    Streaming
4	    2021-10-26T10:15:06	    Streaming
...	    ...	                    ...

La siguiente consulta:

SELECT 
    System.Timestamp() as WindowEndTime, 
    Topic, 
    COUNT(*) AS Count
FROM 
    TwitterStream TIMESTAMP BY CreatedAt
GROUP BY 
    Topic, 
    HoppingWindow(second,10,5)

Se devolverá:

WindowEndTime	        Topic	    Count
2021-10-26T10:15:10	    Streaming	5
2021-10-26T10:15:15	    Streaming	3
2021-10-26T10:15:20	    Streaming	2
2021-10-26T10:15:25	    Streaming	4
2021-10-26T10:15:30	    Streaming	4

In [ ]:
SLIDING WINDOW
==============

Las Sliding windows, a diferencia de las Tumbling o Hopping windows, emiten eventos sólo para los puntos en el tiempo en los que el 
contenido de la ventana cambia realmente. En otras palabras, cuando un evento entra o sale de la ventana. Por lo tanto, cada ventana 
tiene al menos un evento. Al igual que en las Hopping windows, los eventos pueden pertenecer a más de una sliding window.

<center><img src="https://i.postimg.cc/pdtBGPJY/event5.png"></center>

In [ ]:
Con los siguientes datos de entrada (ilustrados más arriba):

Stamp	CreatedAt	            Topic
1	    2021-10-26T10:15:10	    Streaming
5	    2021-10-26T10:15:12	    Streaming
9	    2021-10-26T10:15:15	    Streaming
7	    2021-10-26T10:15:15	    Streaming
8	    2021-10-26T10:15:27	    Streaming

La siguiente consulta:

SELECT 
    System.Timestamp() as WindowEndTime, 
    Topic, 
    COUNT(*) AS Count
FROM 
    TwitterStream TIMESTAMP BY CreatedAt
GROUP BY 
    Topic, 
    SlidingWindow(second,10)
HAVING 
    COUNT(*) >=3

Se devolverá:

WindowEndTime	        Topic	    Count
2021-10-26T10:15:15	    Streaming	4
2021-10-26T10:15:20	    Streaming	3

In [ ]:
SESSION WINDOW
==============

Las funciones Session window agrupan los eventos que llegan en momentos similares, filtrando los periodos de tiempo en los que no hay 
datos. Tiene tres parámetros principales: timeout, maximum duration y partitioning key (opcional).

Una session window comienza cuando se produce el primer evento. Si se produce otro evento dentro del timeout (tiempo de espera) 
especificado desde el último evento ingestado, entonces la ventana se extiende para incluir el nuevo evento. En caso contrario, si no 
se produce ningún evento durante el timeout, la ventana se cierra en ese momento.

Si siguen produciéndose eventos dentro del timeout especificado, la "session window" seguirá extendiéndose hasta que se alcance la 
"maximum duration". Los intervalos de comprobación de la duración máxima (maximum duration) tienen el mismo tamaño que la duración 
máxima especificada. Por ejemplo, si la duración máxima es 10, entonces las comprobaciones sobre si la ventana excede la duración 
máxima ocurrirán en t = 0, 10, 20, 30, etc.

Cuando se proporciona una clave de partición, los eventos se agrupan por la clave y la session window se aplica a cada grupo de forma 
independiente. Esta partición es útil para los casos en los que se necesitan diferentes session windows para diferentes usuarios o 
dispositivos.

<center><img src="https://i.postimg.cc/L5LH9kG1/event6.png"></center>

In [ ]:
Con los siguientes datos de entrada (ilustrados arriba):

Stamp	CreatedAt	            Topic
1	    2021-10-26T10:15:01	    Streaming
2	    2021-10-26T10:15:04	    Streaming
3	    2021-10-26T10:15:13	    Streaming
...	    ...	                    ...

La siguiente consulta:

SELECT System.Timestamp() as WindowEndTime, Topic, COUNT(*) AS Count
FROM TwitterStream TIMESTAMP BY CreatedAt
GROUP BY Topic, SessionWindow(second,5,10)

Se devolverá:

WindowEndTime	        Topic	    Count
2021-10-26T10:15:09	    Streaming	2
2021-10-26T10:15:24	    Streaming	4
2021-10-26T10:15:31	    Streaming	2
2021-10-26T10:15:39	    Streaming	1

In [ ]:
SNAPSHOT WINDOW
===============

Las "snapshot windows" agrupan los eventos que tienen el mismo timestamp. A diferencia de otros tipos de ventanas, que requieren una 
window function específica (como SessionWindow()), puede aplicar una snapshot window añadiendo System.Timestamp() a la cláusula GROUP BY.

<center><img src="https://i.postimg.cc/T3WcRKPX/event7.png"></center>

In [ ]:
Con los siguientes datos de entrada (ilustrados arriba):

Stamp	CreatedAt	            Topic
1	    2021-10-26T10:15:04	    Streaming
2	    2021-10-26T10:15:04	    Streaming
3	    2021-10-26T10:15:04	    Streaming
...	    ...	                    ...

La siguiente consulta:

SELECT System.Timestamp() as WindowEndTime, Topic, COUNT(*) AS Count
FROM TwitterStream TIMESTAMP BY CreatedAt
GROUP BY Topic, System.Timestamp()

Se devolverá:

WindowEndTime	        Topic	    Count
2021-10-26T10:15:04	    Streaming	4
2021-10-26T10:15:10	    Streaming	2
2021-10-26T10:15:13	    Streaming	1
2021-10-26T10:15:22	    Streaming	2